In [ ]:
import numpy as np

class MixedNaiveBayes:
    """
    A Naive Bayes classifier that supports mixed feature types:
    - Gaussian (continuous numerical features)
    - Bernoulli (binary 0/1 features)
    - Categorical (multi-class discrete features)
    - Multinomial (count-based features)

    The classifier computes all likelihoods in log space to avoid
    numerical underflow and applies Laplace smoothing to Bernoulli,
    Categorical, and Multinomial likelihoods.

    Notes
    -----
    - Class priors P(Y=c) are computed WITHOUT Laplace smoothing.
    - Feature likelihoods P(X_j | Y=c) use Laplace smoothing for
      Bernoulli, Categorical, and Multinomial features.
    - Gaussian features use MLE mean and variance.

    Parameters
    ----------
    eps : float, optional
        Small constant added to Gaussian variances to avoid zero variance.
    laplace_alpha : float, optional
        Laplace smoothing parameter for Bernoulli, Categorical, and Multinomial features.

    Example
    -------
    >>> X = np.array([[5.1, 1, "red"],
    ...               [4.9, 0, "blue"],
    ...               [5.0, 1, "red"]])
    >>> y = np.array(["A", "B", "A"])
    >>> feature_types = ["gaussian", "bernoulli", "categorical"]
    >>> model = MixedNaiveBayes()
    >>> model.fit(X, y, feature_types)
    >>> model.predict(X)
    array(['A', 'B', 'A'], dtype='<U1')
    """

    def __init__(self, eps=1e-8, laplace_alpha=1.0):
        """
        Initialize model parameters.

        Parameters
        ----------
        eps : float
            Stability constant added to Gaussian variances.
        laplace_alpha : float
            Laplace smoothing strength for Bernoulli, Categorical, and Multinomial features.

        Example
        -------
        >>> model = MixedNaiveBayes(eps=1e-6, laplace_alpha=1.0)
        """
        self.eps = eps
        self.alpha = laplace_alpha

        self.feature_types = None
        self.classes_ = None

        # learned parameters
        self.log_priors_ = {}
        self.gaussian_means_ = {}
        self.gaussian_vars_ = {}
        self.bernoulli_probs_ = {}
        self.categorical_probs_ = {}
        self.categorical_values_ = {}
        self.multinomial_probs_ = {}

    #  ikelihood functions
    def _gaussian_log_pdf(self, x, mean, var):
        """
        Compute log density of a univariate Gaussian.

        Parameters
        ----------
        x : float
            Observed feature value.
        mean : float
            Gaussian mean for class c and feature j.
        var : float
            Gaussian variance for class c and feature j.

        Returns
        -------
        float
            Log probability density.

        Example
        -------
        >>> model = MixedNaiveBayes()
        >>> model._gaussian_log_pdf(5.0, mean=5.0, var=1.0)
        0.0
        """
        return -0.5 * (np.log(2.0 * np.pi * var) + ((x - mean) ** 2) / var)

    def _bernoulli_log_likelihood(self, x, p):
        """
        Compute log-likelihood for Bernoulli feature.

        Parameters
        ----------
        x : int (0 or 1)
            Observed feature value.
        p : float
            P(X=1 | Y=c), Laplace-smoothed.

        Returns
        -------
        float
            Log-likelihood under Bernoulli distribution.

        Example
        -------
        >>> model = MixedNaiveBayes()
        >>> model._bernoulli_log_likelihood(1, p=0.8)
        -0.223143551...
        """
        return x * np.log(p) + (1.0 - x) * np.log(1.0 - p)

    def _categorical_log_likelihood(self, value, probs_dict):
        """
        Compute log-likelihood of a categorical feature.

        Parameters
        ----------
        value : hashable
            Observed category label.
        probs_dict : dict
            Mapping category -> probability.

        Returns
        -------
        float
            Log-likelihood of observing this category.

        Example
        -------
        >>> probs = {"red": 0.7, "blue": 0.3}
        >>> model = MixedNaiveBayes()
        >>> model._categorical_log_likelihood("red", probs)
        -0.35667494...
        """
        return np.log(probs_dict[value])
    
    def _multinomial_log_likelihood(self, x, probs):
        """
        Log-likelihood for a Multinomial feature.

        Parameters
        ----------
        x : int or float
            Count value for feature j.
        probs : float
            Probability P(feature j | class c), Laplace-smoothed.

        Returns
        -------
        float
            x * log(probability)

        Example
        -------
        >>> model._multinomial_log_likelihood(3, probs=0.2)
        -4.828...
        """
        return x * np.log(probs)

  
    # Aggregator for full likelihood per sample/class
    def _compute_log_likelihood(self, x_row, class_label):
        """
        Compute sum of log-likelihoods across all features for a class.

        Parameters
        ----------
        x_row : array-like of shape (n_features,)
            Single input sample.
        class_label : object
            Class for which likelihood is computed.

        Returns
        -------
        float
            Total log-likelihood for this sample under class c.

        Example
        -------
        >>> model._compute_log_likelihood(["red"], "A")  # with categorical model
        -1.2
        """
        log_l = 0.0

        for j, ftype in enumerate(self.feature_types):
            value = x_row[j]

            if ftype == "gaussian":
                log_l += self._gaussian_log_pdf(
                    x=value,
                    mean=self.gaussian_means_[class_label][j],
                    var=self.gaussian_vars_[class_label][j]
                )

            elif ftype == "bernoulli":
                p = self.bernoulli_probs_[class_label][j]
                log_l += self._bernoulli_log_likelihood(value, p)

            elif ftype == "categorical":
                probs = self.categorical_probs_[class_label][j]
                log_l += self._categorical_log_likelihood(value, probs)

            elif ftype == "multinomial":
                p = self.multinomial_probs_[class_label][j]
                log_l += self._multinomial_log_likelihood(value, p)

        return log_l


    # Fit
    def fit(self, X, y, feature_types):
        """
        Fit Naive Bayes model by estimating:
        - class priors P(Y=c)
        - Gaussian parameters (mean, variance)
        - Bernoulli parameters with Laplace smoothing
        - Categorical parameters with Laplace smoothing
        - Multinomial parameters with Laplace smoothing

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training feature matrix.
        y : array-like of shape (n_samples,)
            Training class labels.
        feature_types : list of str
            List specifying type of each feature:
            "gaussian", "bernoulli", "categorical", or "multinomial".

        Returns
        -------
        self : MixedNaiveBayes
            Trained model.

        Example
        -------
        >>> model = MixedNaiveBayes()
        >>> model.fit(X, y, ["gaussian", "bernoulli"])
        """
        X = np.asarray(X)
        y = np.asarray(y)
        self.feature_types = feature_types
        n_samples, n_features = X.shape

        self.classes_ = np.unique(y)
        multinomial_idx = [j for j, f in enumerate(feature_types) if f == "multinomial"]

        # record categorical possible values
        for j, ftype in enumerate(feature_types):
            if ftype == "categorical":
                self.categorical_values_[j] = np.unique(X[:, j])

        # compute class priors WITHOUT smoothing
        for c in self.classes_:
            Nc = np.sum(y == c)
            self.log_priors_[c] = np.log(Nc / n_samples)

        # initialize dicts
        for c in self.classes_:
            self.gaussian_means_[c] = {}
            self.gaussian_vars_[c] = {}
            self.bernoulli_probs_[c] = {}
            self.categorical_probs_[c] = {}
            self.multinomial_probs_[c] = {}


        # compute likelihood parameters
        for c in self.classes_:
            Xc = X[y == c]
            Nc = len(Xc)

            if len(multinomial_idx) > 0:
                total_multinomial_count = np.sum(Xc[:, multinomial_idx].astype(float))
            else:
                total_multinomial_count = 0

            for j, ftype in enumerate(feature_types):

                # Gaussian MLE
                if ftype == "gaussian":
                    mean = Xc[:, j].mean()
                    var = Xc[:, j].var() + self.eps
                    self.gaussian_means_[c][j] = mean
                    self.gaussian_vars_[c][j] = var

                # Bernoulli likelihood with Laplace smoothing
                elif ftype == "bernoulli":
                    count1 = np.sum(Xc[:, j] == 1)
                    p = (count1 + self.alpha) / (Nc + 2 * self.alpha)
                    self.bernoulli_probs_[c][j] = p

                # Categorical likelihood with Laplace smoothing
                elif ftype == "categorical":
                    values = self.categorical_values_[j]
                    K = len(values)

                    probs = {}
                    for v in values:
                        count_v = np.sum(Xc[:, j] == v)
                        p_v = (count_v + self.alpha) / (Nc + K * self.alpha)
                        probs[v] = p_v

                    self.categorical_probs_[c][j] = probs

                # Multinomial likelihood
                elif ftype == "multinomial":


                    count_j = np.sum(Xc[:, j].astype(float))
                    denom = total_multinomial_count + self.alpha * max(len(multinomial_idx), 1)
                    p_j = (count_j + self.alpha) / denom
                    self.multinomial_probs_[c][j] = p_j

        return self

    # Prediction
    def predict_log_proba(self, X):
        """
        Compute log posterior scores for each class.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Feature matrix for prediction.

        Returns
        -------
        log_probs : ndarray of shape (n_samples, n_classes)
            Raw log posterior scores (not normalized).

        Example
        -------
        >>> logp = model.predict_log_proba(X)
        >>> logp.shape
        (3, 2)
        """
        X = np.asarray(X)
        n_samples = X.shape[0]
        log_probs = np.zeros((n_samples, len(self.classes_)))

        for i in range(n_samples):
            x_row = X[i]
            for k, c in enumerate(self.classes_):
                log_l = self._compute_log_likelihood(x_row, c)
                log_probs[i, k] = self.log_priors_[c] + log_l

        return log_probs

    def predict_proba(self, X):
        """
        Convert log posterior scores into normalized probabilities using
        the log-sum-exp trick to avoid numerical instability.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        probs : ndarray of shape (n_samples, n_classes)
            Normalized class probability distribution.

        Example
        -------
        >>> model.predict_proba(X)
        array([[0.7, 0.3],
               [0.4, 0.6]])
        """
        log_probs = self.predict_log_proba(X)

        max_log = np.max(log_probs, axis=1, keepdims=True)
        shifted = log_probs - max_log
        probs = np.exp(shifted)
        probs = probs / probs.sum(axis=1, keepdims=True)

        return probs

    def predict(self, X):
        """
        Predict the most likely class for each sample.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        labels : ndarray of shape (n_samples,)
            Predicted class labels.

        Example
        -------
        >>> model.predict(X)
        array(['A', 'B'], dtype='<U1')
        """
        probs = self.predict_proba(X)
        class_idx = np.argmax(probs, axis=1)
        return self.classes_[class_idx]
